<a href="https://colab.research.google.com/github/harim061/FakeNewsDetection/blob/main/FINAL/FakeNewsDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
start = time.time()

In [ ]:
path = '/content/drive/MyDrive/fakeData/'

In [ ]:
fake_df = pd.read_csv(path +'Fake.csv')
real_df = pd.read_csv(path +'True.csv')

In [ ]:
fake_df = fake_df[['title', 'text']]
real_df = real_df[['title', 'text']]

In [ ]:
fake_df['check'] = 0
real_df['check'] = 1
df = pd.concat([fake_df, real_df], ignore_index=True, sort=False)

<ipython-input-7-d8a84075019a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_df['check'] = 0


In [ ]:
df.head()

In [ ]:
df.shape

(44898, 3)

In [ ]:
df['title_text'] = df['title'] + ' ' + df['text']
df.drop(['title', 'text'], axis=1, inplace=True)

In [ ]:
df.head()

,check,title_text
0,0,Donald Trump Sends Out Embarrassing New Year’...
1,0,Drunk Bragging Trump Staffer Started Russian ...
2,0,Sheriff David Clarke Becomes An Internet Joke...
3,0,Trump Is So Obsessed He Even Has Obama’s Name...
4,0,Pope Francis Just Called Out Donald Trump Dur...


In [ ]:
def load_data(path):
    fake_df = pd.read_csv(path +'Fake.csv')
    real_df = pd.read_csv(path +'True.csv')

    fake_df = fake_df[['title', 'text']]
    real_df = real_df[['title', 'text']]

    fake_df['check'] = 0
    real_df['check'] = 1
    df = pd.concat([fake_df, real_df], ignore_index=True, sort=False)

    df['title_text'] = df['title'] + ' ' + df['text']
    df.drop(['title', 'text'], axis=1, inplace=True)

    return df

In [ ]:
# 전처리
def preprocessor(text):
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub('-', '', text)
    text = re.sub(r'([.?!])\1+', r'\1', text)  # 여러 번 반복되는 문장 부호 제거
    text = text.lower()  # 모든 문자를 소문자로 변환
    return text

# Lemmatizer 초기화
lemmatizer = WordNetLemmatizer()
# Stopwords 초기화 (불필요한 단어 제거)
stop_words = set(stopwords.words('english'))

In [ ]:
# 전처리와 토큰화
def preprocess_and_tokenize(text):

    sentences = sent_tokenize(text)
    text = re.sub(r'([.?!])\1+', r'\1', text)
    # 모든 문자를 소문자로 변환
    text = text.lower()
    processed_sentences = []
    for sent in sentences:
        # 단어 토큰화
        words = word_tokenize(sent)

        # 소문자화
        words = [word.lower() for word in words]

        # 불필요한 단어 및 특수문자 제거
        words = [word for word in words if word.isalnum() and word not in stop_words]

        # Lemmatization (기본형으로 변환)
        words = [lemmatizer.lemmatize(word) for word in words]

        processed_sentences.append(' '.join(words))

    return ' '.join(processed_sentences)


In [ ]:
# TF-IDF 벡터라이저 반환
def get_tfidf_vectorizer():
    return TfidfVectorizer(strip_accents=None,
                           lowercase=False,
                           preprocessor=None,
                           tokenizer=preprocess_and_tokenize,
                           use_idf=True,
                           norm='l2',
                           smooth_idf=True)

In [ ]:
# 유튜브 스크립트 전처리
def preprocess_youtube_script(script_text):
    script_text = preprocess_and_tokenize(script_text)
    return script_text

In [ ]:
# 문맥 기반 오류 검사
nlp = spacy.load("en_core_web_sm")

def context_checker(text):
    doc = nlp(text)
    errors = []
    for i, token in enumerate(doc[:-1]):
        if token.ent_type_ and doc[i+1].ent_type_ and not token.is_punct and not doc[i+1].is_punct:
            errors.append((token, doc[i+1]))

    for error in errors:
        print(f"Potential error: {error[0].text} {error[1].text}")

    return errors

def fix_context_errors(text, errors):
    corrected_text = text
    for error in errors:
        wrong_word = error[0].text
        correct_word = error[1].text  # Assume the second word is correct
        corrected_text = corrected_text.replace(wrong_word, correct_word, 1)

    return corrected_text

def process_and_check_script(script_text):
    cleaned_script = preprocess_youtube_script(script_text)
    context_errors = context_checker(cleaned_script)  # Get context errors
    corrected_script = fix_context_errors(cleaned_script, context_errors)
    return corrected_script

In [ ]:
clf.score(X_test, y_test)

0.9969933184855234

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
y_pred = clf.predict(X_test)
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

In [ ]:
binary_predictions = []

for i in y_pred:
    if i >= 0.5:
        binary_predictions.append(1)
    else:
        binary_predictions.append(0)
import matplotlib.pyplot as plt
import seaborn as sns
matrix = confusion_matrix(binary_predictions, y_test, normalize='all')
plt.figure(figsize=(10, 6))
ax= plt.subplot()
sns.heatmap(matrix, annot=True, ax = ax)

# labels, title and ticks
ax.set_xlabel('Predicted Labels', size=15)
ax.set_ylabel('True Labels', size=15)
ax.set_title('Confusion Matrix', size=15)
ax.xaxis.set_ticklabels([0,1], size=15)
ax.yaxis.set_ticklabels([0,1], size=15);

In [ ]:
end = time.time()
print(end - start)

813.5157783031464


In [ ]:
!pip install translate

In [ ]:
!pip install langdetect

In [ ]:
!pip install googletrans==4.0.0rc1

In [ ]:
from googletrans import Translator

In [ ]:
from langdetect import detect


def text_and_translate(user_title, user_text):

    news_text = user_title + " " + user_text

    translator = Translator()

    if detect(news_text) != 'en':
         news_text = translator.translate(news_text, dest='en').text
    # print(news_text)
    return news_text



def predict_fake_or_real(news_text):

    # clf 객체 로딩
    with open("text_classifier_model.pkl", "rb") as f:
        clf = pickle.load(f)
    with open("tfidf_vectorizer.pkl", "rb") as tfidf_file:
        tfidf = pickle.load(tfidf_file)

    # 전처리 단계
    news_text = process_and_check_script(news_text)
    # TF-IDF 변환
    news_vector = tfidf.transform([news_text])

    # 모델을 사용하여 예측
    prediction = clf.predict(news_vector)
    prediction_proba = clf.predict_proba(news_vector)

    if prediction[0] == 0:  # prediction은 배열 형태로 반환되므로 prediction[0]을 사용
        return "Fake News", prediction_proba[0][0] * 100  # 가짜 뉴스일 확률
    else:
        return "Real News", prediction_proba[0][1] * 100  # 진짜 뉴스일 확률


def explain_prediction(news_text, result):
    # clf 객체 로딩
    with open("text_classifier_model.pkl", "rb") as f:
        clf = pickle.load(f)
    with open("tfidf_vectorizer.pkl", "rb") as tfidf_file:
        tfidf = pickle.load(tfidf_file)

    news_text = process_and_check_script(news_text)
    news_vector = tfidf.transform([news_text])

    print("!!!!" + news_text)
    feature_names = tfidf.get_feature_names_out()
    coefficients = clf.coef_[0]
    words_importance = []

    for idx in news_vector.nonzero()[1]:
        word = feature_names[idx]
        importance = coefficients[idx] * news_vector[0, idx]
        words_importance.append((word, importance))

    words_importance.sort(key=lambda x: abs(x[1]), reverse=True)

    if result == "Fake News":
        return [word for word in words_importance if word[1] < 0][:10]
    else:
        return [word for word in words_importance if word[1] > 0][:10]



while True:

    user_title = input("Please enter a news article title or 0 to exit: ").strip()

    if user_title == '0':
        print("Exiting the program.")
        break

    user_text = input("Please enter the news article text: ").strip()
    news_text = text_and_translate(user_title, user_text)

    result, probability = predict_fake_or_real(news_text)
    explanation = explain_prediction(news_text, result)


    print("This news is:", result)
    print(f"Probability: {probability:.2f}%")
    print("Top 10 influencing words:")
    for word, importance in explanation:
        print(f"{word}: {'supports Fake' if importance < 0 else 'supports Real'} with weight {abs(importance)}")